<a href="https://colab.research.google.com/github/syauqyideas/DualSPHysic_Colab_Notebook_Runner/blob/main/OpenFoam_COLAB_Public.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1><strong> Set Up Your Case
<h3> Note That it is recommended to prepare your case in your local computer before sending it to colab
<h5> Actually <strong>Colab is not proper place to run High CPU Dependent App (like Openfoam)</strong>, but since it give you at least good Processor, and 87 hour of free runtime, it still worth to try   
<br>
<p>*consider to buy colab premium if you can afford it

In [ ]:
import os

name = 'name' #@param {type: 'string'}
CaseClassification = 'SinglePhase' #@param ['Multiphase','SinglePhase','Combustion']
if CaseClassification == 'SinglePhase':
  solverSinglePhase = 'rhoPimpleFoam' #@param['NaN','simpleFoam','icoFoam','pimpleFoam','rhoPimpleFoam']
  os.environ['solver'] = solverSinglePhase
elif CaseClassification == 'Multiphase':
  solverMultiphase = 'interCondensatingEvaporatingFoam' #@param['interFoam','interCondensatingEvaporatingFoam']
  os.environ['solver'] = solverMultiphase
elif CaseClassification == 'Combustion':
  solverCombustion = 'XiEngineFoam' #@param['XiEngineFoam','engineFoam']
  os.environ['solver'] = solverCombustion
os.environ['name'] = name
OpenFoamVersion = '2412' #@param ['2312','2306','2406','2412']{allow-input : true}
#@markdown <h5> consider that author still not tested the openfoam.org version, since it make major update on of 10 <br>

if OpenFoamVersion is not None :
  os.environ['openfoamActivator'] = f'source /usr/lib/openfoam/openfoam{OpenFoamVersion}/etc/bashrc'
  os.environ['FOAM_RUN_COLAB_NAME'] = f'OpenFOAM/user-v{OpenFoamVersion}/run/{name}'
  os.environ['FOAM_RUN_COLAB'] = f'OpenFOAM/user-v{OpenFoamVersion}/run/'
  #os.environ["LD_LIBRARY_PATH"] = "/content/DualSPHysics_Linux-main/src/lib/linux_gcc"# + os.environ.get("LD_LIBRARY_PATH", "")
else:
  print('Insert your Openfoam Version')

#@markdown <br>If you want to use gdrive as your case file source. note that this can be used for continuing case, if you want to continue case running (the openfoam continuity still developing)
is_gdrive_based = True #@param {type: 'boolean'}
OpenFoamDrivePath = 'OpenFoam' #@param {type: 'string'}
if is_gdrive_based == True and OpenFoamDrivePath != None :
  os.environ['gdrive_path'] = f'/content/drive/MyDrive/{OpenFoamDrivePath}/{name}'
#@markdown note that the name of folder case is <strong>need to be the same with "name "variable</strong>

#@markdown if you use github to save your case
github_project = False #@param {type: 'boolean'}
github_project_PAT = False #@param {type: 'boolean'}
personal_project_repo = "" #@param {type: 'string'}
github_username = "" #@param {type: 'string'}
if github_project_PAT == True :
  github_personal_token = "" #@param {type: 'string'}

#if you use personal .sh file :
personal_sh_file = False #@param {type : 'boolean'}
if personal_sh_file == True :
  FileRunnerName = "" #@param {type: 'string'}

run_tutorial = False #@param ['pitzdaily',''] {type: 'boolean'}
if run_tutorial == True :
  tutorial_name = "" #@param {type: 'string'}

<h2><strong>Connect Google Drive

In [ ]:
from google.colab import files
import os

if is_gdrive_based == True :
  from google.colab import drive
  drive.mount('/content/drive')

<h2><strong> Install OpenFOAM

In [ ]:
## Add the repository
!curl -s https://dl.openfoam.com/add-debian-repo.sh | sudo bash

# Update the repository information
!sudo apt-get update

# Install preferred package. Eg,
!sudo apt-get install openfoam{OpenFoamVersion}-default

!pip install IPython

import IPython #at some point, i just bored to see the dirty output
from IPython.display import clear_output

#clear_output()

<h3>Set Up some packages (Oprional)
<h3>Add to some Library to environment (Optional for Editing and Multiprocessing)

In [ ]:
%cd /home
!apt update && upgrade
!apt install nano
!pip install colab-xterm
%load_ext colabxterm
!apt install nano
!apt install mpi
!pip install vtk
clear_output()

<h2><strong>Set Up your OpenFoam Case

In [ ]:
#!$openfoamActivator &&\mkdir -p $FOAM_RUN
!rm -r /content/$name
if is_gdrive_based == True and OpenFoamDrivePath != None :
  !cp -r $gdrive_path /content
  %cd /content/$name

if github_project == True :
  if github_project_PAT == True :
    !rm -r ./{personal_project_repo}
    !git clone https://{github_personal_token}@github.com/{github_username}/{personal_project_repo}.git
  else :
    !git clone https://github.com/{github_username}/{public_project_repo}.git
    !unzip main.zip
    clear_output()
if run_tutorial == True :
  !$openfoamActivator && \
  cp -p $FOAM_TUTORIALS/{tutorial_name} $FOAM_RUN\

<h2><strong>Run Simulation

<h3>SetFields (if you run multiphase)

In [ ]:
#%cd content/$name
%cd $gdrive_path

!$openfoamActivator &&\
setFields

<h3> DecomposePar - to Run multiprocessor (2 processor in google colab) </h3>

In [ ]:
%cd /content/$name
#%cd $gdrive_path

!$openfoamActivator &&\
decomposePar -force

clear_output()

In [ ]:
!$openfoamActivator &&\
    mpirun --allow-run-as-root --oversubscribe -n 2 renumberMesh -overwrite -parallel
clear_output()

<h3>Run Simulation

In [ ]:
!$openfoamActivator &&\
    mpirun --allow-run-as-root --oversubscribe -n 2 $solver -parallel
#clear_output()

In [ ]:
!$openfoamActivator &&\
     $solver
#clear_output()

In [ ]:
if is_gdrive_based == True and OpenFoamDrivePath != None :
  !mkdir -p $gdrive_path/simulated
  !cp -r /content/$name/processor* $gdrive_path/


<h3> Nano Text Editor - actually deprecated, colab is provided its own terminal for free now

In [ ]:
%xterm

In [ ]:
!du -h /content/solverSkripsi/

<h2><strong>Download Simulation Files

In [ ]:
# Get the current working directory
%cd /content
!zip -r $name.zip /content/$name
files.download(f'{name}.zip')


<h3>Make VTK for Internal Analysis (in Development)</h3>

In [ ]:
%cd /content/$name
!$openfoamActivator &&\
reconstructPar

In [ ]:
!$openfoamActivator &&\
foamToVTK -legacy

<h3>Internal Analysis (in Development)</h3>

In [ ]:
def convert_vtm_to_vtk(input_vtm_path, output_base_name):
    """
    Mengonversi setiap blok data dalam file .vtm menjadi file .vtk terpisah.

    Args:
        input_vtm_path (str): Jalur ke file .vtm yang akan dikonversi.
        output_base_name (str): Nama dasar untuk file .vtk keluaran.
    """
    try:
        # 1. Baca file .vtm
        reader = vtk.vtkXMLMultiBlockDataReader()
        reader.SetFileName(input_vtm_path)
        reader.Update()

        multi_block_data = reader.GetOutput()
        if not multi_block_data:
            print(f"Gagal membaca file: {input_vtm_path}")
            return

        num_blocks = multi_block_data.GetNumberOfBlocks()
        print(f"File VTM berisi {num_blocks} blok. Mengonversi...")

        # 2. Iterasi setiap blok dan tulis sebagai file .vtk terpisah
        for i in range(num_blocks):
            block = multi_block_data.GetBlock(i)

            # Pastikan blok tidak kosong
            if block is None:
                print(f"  Blok {i} kosong, dilewati.")
                continue

            # Gunakan vtkDataSetWriter untuk menulis format .vtk lama
            writer = vtk.vtkDataSetWriter()
            writer.SetInputData(block)

            output_file_name = f"{output_base_name}_{i}.vtk"
            writer.SetFileName(output_file_name)
            writer.Write()

            print(f"  Blok {i} berhasil disimpan ke {output_file_name}")

        print("\nKonversi selesai.")

    except Exception as e:
        print(f"Terjadi kesalahan saat konversi: {e}")

# Panggil fungsi dengan jalur file .vtm Anda
# Ganti 'contoh.vtm' dengan jalur file Anda yang sebenarnya


In [ ]:
filename = f"/content/{name}/VTK/{name}_10730.vtk"
reader = vtk.vtkXMLMultiBlockDataReader()
reader.SetFileName(filename)
reader.Update()
data = reader.GetOutput()